## Scraping the data from Wikipedia page

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
from bs4 import BeautifulSoup
import lxml.etree
soup = BeautifulSoup(website_url,'html.parser')
#print(soup.prettify())

In [3]:
table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

In [4]:
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

In [5]:
df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows

In [6]:
df.head()

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [7]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)

### Combined into one row with the neighborhoods separated with a comma

In [8]:
df1=df.groupby("PostalCode").agg(lambda x:','.join(set(x)))

### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [9]:
df1.loc[df1['Neighbourhood']=="Not assigned",'Neighbourhood']=df1.loc[df1['Neighbourhood']=="Not assigned",'Borough']
df1.head()

,Borough,Neighbourhood
PostalCode,,
M1B,Scarborough,"Malvern,Rouge"
M1C,Scarborough,"Rouge Hill,Highland Creek,Port Union"
M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [10]:
df1.shape

(103, 2)